In [3]:
import requests
from lxml import html
import re
import string

In [2]:
detail_url = "https://sh.fang.lianjia.com/loupan/p_aafip/"
detail_res = requests.get(detail_url)
detail_res.status_code

200

In [4]:
detail_tree = html.fromstring(detail_res.text)

In [64]:
detail_texts = detail_tree.xpath("//div[@id='house-details']//text()")

jiage = detail_tree.xpath("//p[@class='jiage']//text()")

clean_whitespace = re.compile(r'[ \t\n\r\x0b\x0c]*')

details = [clean_whitespace.sub('',feild) for feild in detail_texts if clean_whitespace.sub('',feild)]

price = [clean_whitespace.sub('',feild) for feild in jiage if clean_whitespace.sub('',feild)]

In [65]:
details

['楼盘详情',
 '项目地址：',
 '闵行-都会路3799号',
 '售楼处地址：',
 '金都路2888号（近金都路）（接待时间9:30-15:00）',
 '开发商：',
 '上海金色紫都房地产有限公司',
 '物业公司：',
 '星河湾物业管理有限公司',
 '最新开盘：',
 '2016年05月28日',
 '物业类型：',
 '住宅',
 '交房时间：',
 '2018年03月',
 '容积率：',
 '1.40',
 '产权年限：',
 '70年',
 '绿化率：',
 '35%',
 '规划户数：',
 '854',
 '物业费用：',
 '4.5元/m²/月',
 '车位情况：',
 '地下车位数1078',
 '供暖方式：',
 '暂无',
 '供水方式：',
 '民水',
 '供电方式：',
 '民电',
 '建筑类型：',
 '板楼',
 '嫌恶设施：',
 '暂无',
 '占地面积：',
 '714,300㎡',
 '建筑面积：',
 '653,000㎡',
 '查看更多']

In [66]:
price

['均价', '76500', '元/平', '别名/闵行星河湾']

In [71]:
name = detail_tree.xpath("//a[@class='clear']")[0]

In [72]:
name.get('title')

'上海星河湾'

# 楼盘列表

In [47]:
index_url = 'https://sh.fang.lianjia.com/loupan/pg1/'
index_res = requests.get(index_url)
index_res.status_code

200

In [48]:
index_tree = html.fromstring(index_res.text)

In [54]:
loupan_tags_element = index_tree.xpath('//a[@class="name"]')

In [53]:
len(loupan_tags)

10

In [55]:
loupan_tags = [e.get('href') for e in loupan_tags_element]

In [57]:
loupan_tags

['/loupan/p_aafip/',
 '/loupan/p_xcssabnih/',
 '/loupan/p_aapjb/',
 '/loupan/p_ljssyzabney/',
 '/loupan/p_shxhzxxxmdabnfo/',
 '/loupan/p_aaffm/',
 '/loupan/p_aaghz/',
 '/loupan/p_aahck/',
 '/loupan/p_aagzb/',
 '/loupan/p_aahap/']

In [58]:
detail_url_root = "https://sh.fang.lianjia.com"
detail_url_list = [detail_url_root + tag for tag in loupan_tags]

In [59]:
index_url_root = 'https://sh.fang.lianjia.com/loupan/pg{}/'

In [63]:
index_url_root.format(1)

'https://sh.fang.lianjia.com/loupan/pg1/'

# 二手房

### 每条二手房信息抓取

In [104]:
ershou_url = 'https://sh.lianjia.com/ershoufang/pg1/'
res = requests.get(ershou_url)

In [105]:
res.status_code

200

In [106]:
tree = html.fromstring(res.text)

e = tree.xpath("//div[@class='info clear']")

In [208]:
t = e[0].xpath(".//text()")

In [209]:
t

['11号线御桥路站+总价zui低两房+满五唯一+对口建平',
 '地杰国际城E欧泊时光 ',
 ' | 2室2厅 | 99.4平米 | 南 | 毛坯 | 有电梯',
 '低楼层(共18层)2007年建板楼  -  ',
 '御桥',
 '70人关注 / 共114次带看 / 一年前发布',
 '距离11号线御桥站515米',
 '房本满五年',
 '随时看房',
 '615',
 '万',
 '单价61872元/平米']

In [210]:
t = ';'.join(t)
t

'11号线御桥路站+总价zui低两房+满五唯一+对口建平;地杰国际城E欧泊时光 ; | 2室2厅 | 99.4平米 | 南 | 毛坯 | 有电梯;低楼层(共18层)2007年建板楼  -  ;御桥;70人关注 / 共114次带看 / 一年前发布;距离11号线御桥站515米;房本满五年;随时看房;615;万;单价61872元/平米'

In [258]:
Title = ['CommunityName','RoomType','Area(m2)','Pre-Price','Total-Price','Attention','Seen']

In [273]:
def parse_2shoufang_info(element_text):
    """解析二手房信息元素"""
    #logging.info('element_text:\n{}'.format(';'.join(element_text)))
    try:
        if '房主自荐' in element_text:
            flag_index = element_text.index('房主自荐') + 1        
        elif '新上' in element_text:
            flag_index = element_text.index('新上') + 1
        else:
            flag_index = 1
        CommunityName = element_text[flag_index].strip()      
        #logging.info('element_text[2]{}'.format(element_text[2]))
        RoomType = element_text[flag_index + 1].strip().split('|',3)[1].strip()
        Area = element_text[flag_index + 1].strip().split('|',3)[2][:-3]
        Pre_Price = re.search(r'[\d\.]+',element_text[-1]).group(0)
        Total_Price = element_text[-3]
        #logging.info('element_text[5]{}'.format(element_text[5]))
        for feild in element_text:
            if '人关注' in feild:    
                Attention,Seen = re.findall(r'[\d]+',feild)[:2]
        
        return [CommunityName,RoomType,Area,Pre_Price,Total_Price,Attention,Seen]
    except Exception as e:
        print(e)
        #logging.info('信息解析有误,报错{}'.format(e))
        #logging.info('解析错误：{}'.format(element_text))
        return None

In [111]:
td = ' | 2室2厅 | 99.4平米 | 南 | 毛坯 | 有电梯'

In [114]:
td.strip().split('|',3)[1]

True

In [149]:
tt = '70人关注 / 共114次带看 / 一年前发布'

In [154]:
re.findall(r'[\d\.]+',tt)

['70', '114']

In [205]:
ershoufang_info(t)

['地杰国际城E欧泊时光', ' 2室2厅 ', ' 99.4平米 ', '61872', '70', '114']

In [221]:
t_zhabei_url = 'https://sh.lianjia.com/ershoufang/zhabei/'
t_zhabei_res = requests.get(t_zhabei_url)
t_zhabei_tree = html.fromstring(t_zhabei_res.text)
t_zhabei_e = t_zhabei_tree.xpath("//div[@class='info clear']")
t_zhabei = t_zhabei_e[3].xpath(".//text()")

In [213]:
t_zhabei
['11号线御桥路站+总价zui低两房+满五唯一+对口建平',
 '地杰国际城E欧泊时光 ',
 ' | 2室2厅 | 99.4平米 | 南 | 毛坯 | 有电梯',
 '低楼层(共18层)2007年建板楼  -  ',
 '御桥',
 '70人关注 / 共114次带看 / 一年前发布',
 '距离11号线御桥站515米',
 '房本满五年',
 '随时看房',
 '615',
 '万',
 '单价61872元/平米']

['稀 缺房源 中高区大三房 带车位 带学 区 去名急售',
 '慧芝湖花园 ',
 ' | 3室2厅 | 144平米 | 西南 | 精装 | 有电梯',
 '中楼层(共36层)2009年建板楼  -  ',
 '大宁',
 '9人关注 / 共11次带看 / 26天以前发布',
 '距离1号线上海马戏城站917米',
 '房本满五年',
 '1320',
 '万',
 '单价91667元/平米']

In [251]:
parse_2shoufang_info(t_zhabei)

['彭五小区', ' 2室0厅 ', ' 48.02平米 ', '53728', '112', '76']

In [255]:
t0 = t_zhabei

In [252]:
t1 = ['乍浦东方巴黎城 5室3厅 370万', '新上', '乍浦东方巴黎城 ', ' | 5室3厅 | 363.6平米 | 南 | 其他', '低楼层(共4层)暂无数据  -  ', '嘉兴', '0人关注 / 共0次带看 / 1天以前发布', '房本满五年', '370', '万', '单价10177元/平米']

In [253]:
t2 = ['花桥绿地21城D区孝贤坊 2室1厅 175万', '新上', '房主自荐', '花桥绿地21城D区孝贤坊 ', ' | 2室1厅 | 81.26平米 | 南 | 精装', '高楼层(共11层)暂无数据  -  ', '昆山', '7人关注 / 共0次带看 / 2天以前发布', '175', '万', '单价21536元/平米']

In [254]:
t3 = ['长泰淀湖观园 4室2厅 500万', '房主自荐', '长泰淀湖观园 ', ' | 4室2厅 | 229.66平米 | 南 | 精装', '低楼层(共1层)暂无数据  -  ', '昆山', '1人关注 / 共0次带看 / 1个月以前发布', '500', '万', '单价21772元/平米']

In [268]:
parse_2shoufang_info(t0)

name 'Pre' is not defined


In [274]:
for t in (t0,t1,t2,t3):
    print(parse_2shoufang_info(t))

['彭五小区', '2室0厅', ' 48.02', '53728', '258', '112', '76']
['乍浦东方巴黎城', '5室3厅', ' 363.6', '10177', '370', '0', '0']
['花桥绿地21城D区孝贤坊', '2室1厅', ' 81.26', '21536', '175', '7', '0']
['长泰淀湖观园', '4室2厅', ' 229.66', '21772', '500', '1', '0']


### 获得二手房区域列表

In [155]:
ERSHOUFANG_URL_ROOT = 'https://sh.lianjia.com/ershoufang/'
root_res = requests.get(ERSHOUFANG_URL_ROOT)

In [156]:
root_res.status_code

200

In [188]:
def get_areas_info(response):
    """提取页面中区域名称链接"""
    area_elements = html.fromstring(root_res.text).xpath('//div[@data-role="ershoufang"]/div//a[@href]')
    areas_info = {}
    for area_element in area_elements:
        areas_info[area_element.text.strip()] = area_element.get('href').split('/')[-2] 
    return areas_info

In [240]:
area_url = 'https://sh.lianjia.com/ershoufang/songjiang/'
area_res = requests.get(area_url)

def get_subareas_info(area_res):
    """提取页面中子区域名称链接"""
    subarea_elements = html.fromstring(area_res.text).xpath('//div[@data-role="ershoufang"]/div[2]//a[@href]')
    subareas_info = {}
    for subarea_element in subarea_elements:
        subareas_info[subarea_element.text.strip()] = subarea_element.get('href').split('/')[-2] 
    return subareas_info

subareas_info = get_subareas_info(area_res)
subareas_info

{'九亭': 'jiuting',
 '佘山': 'sheshan',
 '叶榭': 'yexie',
 '小昆山': 'xiaokunshan',
 '新桥': 'xinqiao',
 '新浜': 'xinbang',
 '松江大学城': 'songjiangdaxuecheng',
 '松江新城': 'songjiangxincheng',
 '松江老城': 'songjianglaocheng',
 '泖港': 'maogang',
 '泗泾': 'sijing',
 '石湖荡': 'shihudang',
 '莘闵别墅': 'shenminbieshu',
 '车墩': 'chedun'}

In [241]:
subarea_over100p,subarea_in100p = pick_over100p_area(subareas_info)

err urlhttps://sh.lianjia.com/ershoufang/maogang/
err urlhttps://sh.lianjia.com/ershoufang/shihudang/
err urlhttps://sh.lianjia.com/ershoufang/xiaokunshan/
err urlhttps://sh.lianjia.com/ershoufang/xinbang/


In [244]:
subarea_in100p

{'九亭': ['25', 'https://sh.lianjia.com/ershoufang/jiuting/'],
 '佘山': ['4', 'https://sh.lianjia.com/ershoufang/sheshan/'],
 '叶榭': ['1', 'https://sh.lianjia.com/ershoufang/yexie/'],
 '新桥': ['9', 'https://sh.lianjia.com/ershoufang/xinqiao/'],
 '松江大学城': ['15', 'https://sh.lianjia.com/ershoufang/songjiangdaxuecheng/'],
 '松江新城': ['15', 'https://sh.lianjia.com/ershoufang/songjiangxincheng/'],
 '松江老城': ['19', 'https://sh.lianjia.com/ershoufang/songjianglaocheng/'],
 '泗泾': ['26', 'https://sh.lianjia.com/ershoufang/sijing/'],
 '莘闵别墅': ['16', 'https://sh.lianjia.com/ershoufang/shenminbieshu/'],
 '车墩': ['3', 'https://sh.lianjia.com/ershoufang/chedun/']}

In [196]:
areas_info = get_areas_info(root_res)
areas_info

{'上海周边': 'shanghaizhoubian',
 '嘉定': 'jiading',
 '奉贤': 'fengxian',
 '宝山': 'baoshan',
 '崇明': 'chongming',
 '徐汇': 'xuhui',
 '普陀': 'putuo',
 '杨浦': 'yangpu',
 '松江': 'songjiang',
 '浦东': 'pudong',
 '虹口': 'hongkou',
 '金山': 'jinshan',
 '长宁': 'changning',
 '闵行': 'minhang',
 '闸北': 'zhabei',
 '青浦': 'qingpu',
 '静安': 'jingan',
 '黄浦': 'huangpu'}

In [184]:
def get_total_page(response):
    """获取区域信息对应的页面总数"""
    total_page_element = html.fromstring(response.text).xpath('//div[@class="page-box house-lst-page-box"]')[0]
    return re.match(r'^\{\"totalPage\":(\d+).*\}$',total_page_element.get('page-data')).group(1)
    

In [238]:
def pick_over100p_area(areas_info):
    """以1面总数是否超100对区域分类"""
    area_in100p = {}
    area_over100p = {}
    for area_name,area_url in areas_info.items():
        area_url = ERSHOUFANG_URL_ROOT + area_url + '/'
        try:
            response = requests.get(area_url)
            total_page = get_total_page(response)
            areas_info[area_name] = [total_page,area_url]
        except:
            print('err url{}'.format(area_url))
            continue
        if int(total_page) >= 100:
            area_over100p[area_name] = [total_page,area_url]
        else:
            area_in100p[area_name] = [total_page,area_url]
    return area_over100p,area_in100p

In [199]:
area_over100p,area_in100p = pick_over100p_area(areas_info)

In [200]:
area_over100p

{'嘉定': ['100', 'https://sh.lianjia.com/ershoufang/jiading/'],
 '宝山': ['100', 'https://sh.lianjia.com/ershoufang/baoshan/'],
 '徐汇': ['100', 'https://sh.lianjia.com/ershoufang/xuhui/'],
 '普陀': ['100', 'https://sh.lianjia.com/ershoufang/putuo/'],
 '杨浦': ['100', 'https://sh.lianjia.com/ershoufang/yangpu/'],
 '松江': ['100', 'https://sh.lianjia.com/ershoufang/songjiang/'],
 '浦东': ['100', 'https://sh.lianjia.com/ershoufang/pudong/'],
 '闵行': ['100', 'https://sh.lianjia.com/ershoufang/minhang/']}

In [201]:
area_in100p

{'上海周边': ['2', 'https://sh.lianjia.com/ershoufang/shanghaizhoubian/'],
 '奉贤': ['73', 'https://sh.lianjia.com/ershoufang/fengxian/'],
 '崇明': ['2', 'https://sh.lianjia.com/ershoufang/chongming/'],
 '虹口': ['94', 'https://sh.lianjia.com/ershoufang/hongkou/'],
 '金山': ['3', 'https://sh.lianjia.com/ershoufang/jinshan/'],
 '长宁': ['91', 'https://sh.lianjia.com/ershoufang/changning/'],
 '闸北': ['94', 'https://sh.lianjia.com/ershoufang/zhabei/'],
 '青浦': ['77', 'https://sh.lianjia.com/ershoufang/qingpu/'],
 '静安': ['31', 'https://sh.lianjia.com/ershoufang/jingan/'],
 '黄浦': ['56', 'https://sh.lianjia.com/ershoufang/huangpu/']}

In [249]:
wantiguan = 'https://sh.lianjia.com/ershoufang/wantiguan/'
response = requests.get(wantiguan)
get_total_page(response)

'7'

In [280]:
xin_file = 'sh_xin_loupan_details.csv'

In [279]:
import os
import csv
os.getcwd()

'/home/gk07/Instances/Python-Pratices/lianjia_shanghai'

In [313]:
f = open(xin_file,'r')
l = f.readline()

In [307]:
TITLE_XIN = ['Name','Address','Productor','PorpertyCompany','Start-Time','Type','Delivery-Time','Plot-Ratio','LimitOfusingTime','Green-Rate','Planning-homesNum','Property-fee','Floor-Area','Building-Area','Price','DanWei']

In [318]:
def parse_xin_detail(line):
    Name,other = line.split(',',1)
    des_text,price_text = eval(other)
    des_text = eval(des_text)[1::2]
    Address = des_text[0]
    Productor,PorpertyCompany = des_text[2:4]
    for x in '年月日':
        des_text[4] = des_text[4].replace(x,'-')
        des_text[6] = des_text[6].replace(x,'-')
    Start_Time = des_text[4].strip()
    Delivery_Time = des_text[6].strip()
    Type = des_text[5]
    if re.search(r'([\d\.]+)',des_text[7]):
        Plot_Ratio = re.search(r'([\d\.]+)',des_text[7]).group(1)
    else:
        Plot_Ratio = ''
    LimitOfusingTime = des_text[8]
    if re.search(r'([\d\.]+%)',des_text[9]):
        Green_Rate = re.search(r'([\d\.]+%)',des_text[9]).group(1)
    else:
        Green_Rate = ''
    if re.search(r'([\d]+)',des_text[10]):
        Planning_homesNum = re.search(r'([\d]+)',des_text[10]).group(1)
    else:
        Planning_homesNum = ''
    if re.search(r'([\d\.]+)',des_text[11]):
        Property_fee = re.search(r'([\d\.]+)',des_text[11]).group(1)
    else:
        Property_fee = ''
    des_text[-2] = des_text[-2].replace(',','')
    if re.search(r'([\d]+)',des_text[-2]):
        Floor_Area = re.search(r'([\d]+)',des_text[-2]).group(1)
    else:
        Floor_Area = ''
    if re.search(r'([\d]+)',des_text[-1]):
        Building_Area = re.search(r'([\d]+)',des_text[-1]).group(1)
    else:
        Building_Area = ''
    try:
        price_list = eval(price_text)
    except Exception:
        price_list = []
    if len(price_list) == 2:
        Price = price_list[1]
        DanWei = '元/平米'
    elif len(price_list) == 3:
        Price,DanWei = price_list[1:]
    else:
        Price = ''
        DanWei = ''
    return [Name,Address,Productor,PorpertyCompany,Start_Time,
            Type,Delivery_Time,Plot_Ratio,LimitOfusingTime,
            Green_Rate,Planning_homesNum,Property_fee,Floor_Area,
            Building_Area,Price,DanWei]

In [319]:
parse_xin_detail(l)

['禹洲府',
 '富国路199弄',
 '上海灏溢房地产开发有限公司',
 '禹洲物业服务有限公司上海分公司',
 '2017-06-27-',
 '住宅',
 '2019-03-01-',
 '2.20',
 '70年',
 '35%',
 '1042',
 '4.5',
 '41262',
 '131',
 '350',
 '万/套']

In [321]:
tl = '森兰名轩,"['项目地址：', '张扬北路东高路路口', '售楼处地址：', '兰谷路2500弄（接待时间9:0-15:0）', '开发商：', '上海同运置业有限公司', '物业公司：', '上海实业', '最新开盘：', '2017年04月12日', '物业类型：', '住宅', '交房时间：', '预计2018年04月', '容积率：', '1.20', '产权年限：', '70年', '绿化率：', '35%', '规划户数：', '426', '物业费用：', '3.9元/m²/月', '车位情况：', '地下车位数600', '供暖方式：', '暂无', '供水方式：', '民水', '供电方式：', '民电', '建筑类型：', '板楼', '嫌恶设施：', '暂无', '占地面积：', '53,000㎡', '建筑面积：', '95,000㎡']",[]'

SyntaxError: invalid character in identifier (<ipython-input-321-53da0c14e2b4>, line 1)